In [81]:
import pandas as pd

df = pd.read_csv('signals_optimal_truth_labeled.csv')
df.columns = ['ticker', 'price', 'time', 'meta_signal', 'meta_previous', 
              '4h', '1d','1w', '1m', 
              '4hp', '1dp', '1wp', '1mp', 
              '4h_rsi', '4h_stochrsi','4h_stoch', '4h_ult', '4h_macd', '4h_hull', 
              '1d_rsi', '1d_stochrsi','1d_stoch', '1d_ult', '1d_macd', '1d_hull', 
              '1w_rsi', '1w_stochrsi','1w_stoch', '1w_ult', '1w_macd', '1w_hull', 
              '1m_rsi', '1m_stochrsi','1m_stoch', '1m_ult', '1m_macd', '1m_hull',
              "4h_divergence","1d_divergence","1w_divergence","1m_divergence",
              'meta_move', 
              'Meta OK','Meta Good', 'Meta ⬆️', 'Meta ⬆️⬆️', 'Meta Bad', 'Meta ⬇️', 'Meta ⬇️⬇️',
              'RSI Over Short', 'RSI Over Long', 'RSI Over All', 'RSI Under Short','RSI Under Long', 'RSI Under All', 
              'StochRSI Over Short','StochRSI Over Long', 'StochRSI Over All', 'StochRSI Under Short','StochRSI Under Long', 'StochRSI Under All', 
              'Stoch Over Short','Stoch Over Long', 'Stoch Over All', 'Stoch Under Short','Stoch Under Long', 'Stoch Under All', 
              'Ult Over Short','Ult Over Long', 'Ult Over All', 'Ult Under Short', 'Ult Under Long', 'Ult Under All',
              'buy', 'sell', 'hold'
]
df.head(1)

,ticker,price,time,meta_signal,meta_previous,4h,1d,1w,1m,4hp,...,Stoch Under All,Ult Over Short,Ult Over Long,Ult Over All,Ult Under Short,Ult Under Long,Ult Under All,buy,sell,hold
0,AMEX-BTG,2.24,1535032150741,-1,0,-1,-1,-1,-1,0.0,...,0,0,0,0,0,0,0,0,0,1


In [82]:
data = df.drop(['ticker', 'time'], axis=1)
data.head(5)

,price,meta_signal,meta_previous,4h,1d,1w,1m,4hp,1dp,1wp,...,Stoch Under All,Ult Over Short,Ult Over Long,Ult Over All,Ult Under Short,Ult Under Long,Ult Under All,buy,sell,hold
0,2.240,-1,0,-1,-1,-1,-1,0.0,-2.0,0.0,...,0,0,0,0,0,0,0,0,0,1
1,2.235,-1,0,-1,-1,-1,-1,0.0,-2.0,0.0,...,0,0,0,0,0,0,0,0,0,1
2,2.210,-1,0,-1,-1,-1,-1,0.0,-2.0,0.0,...,0,0,0,0,0,0,0,1,0,0
3,2.215,-1,0,-1,-1,-1,-1,0.0,-2.0,0.0,...,0,0,0,0,0,0,0,0,0,1
4,2.230,-1,-1,-2,-1,-1,-1,-1.0,-2.0,0.0,...,0,0,0,0,0,0,0,0,1,0


In [83]:
# samples
X = df.drop(['ticker', 'time', 'buy', 'sell', 'hold'], axis=1)
X.head(1)

,price,meta_signal,meta_previous,4h,1d,1w,1m,4hp,1dp,1wp,...,Stoch Over All,Stoch Under Short,Stoch Under Long,Stoch Under All,Ult Over Short,Ult Over Long,Ult Over All,Ult Under Short,Ult Under Long,Ult Under All
0,2.24,-1,0,-1,-1,-1,-1,0.0,-2.0,0.0,...,0,0,1,0,0,0,0,0,0,0


In [84]:
# truth labels (Multiclass Classification)
Y = df.filter(['buy','sell','hold'], axis=1)
Y.head(1)

,buy,sell,hold
0,0,0,1


In [85]:
import numpy as np
import sklearn
import matplotlib
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
# http://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_curve.html
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, r2_score, precision_recall_curve, mean_squared_error

from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

In [39]:
from sklearn.model_selection import train_test_split
# 75%/25% train/test
X_train, X_test, Y_train, Y_test = train_test_split(
    X,Y
)
print('X_train',len(X_train),'examples')
print('X_test',len(X_test),'examples')
print('Y_train',len(Y_train),'examples')
print('Y_test',len(Y_test),'examples')
print('shape', X_train.shape)

X_train 6291 examples
X_test 2097 examples
Y_train 6291 examples
Y_test 2097 examples
shape (6291, 71)


In [50]:
scaler = StandardScaler()  
# Don't cheat - fit only on training data
scaler.fit(X_train)  
X_train = scaler.transform(X_train)  
# apply same transformation to test data
X_test = scaler.transform(X_test)

X_train[0]

array([-0.20201881, -0.27460495, -0.2560588 , -1.2323489 , -0.99940778,
       -0.79006888,  0.81802391, -0.10914232, -1.58824183, -0.20990514,
        0.04777959, -0.51033325,  1.18058347,  0.74828177, -0.03582549,
        0.67895865, -0.85706286,  0.24440588, -1.17524181, -0.3237314 ,
        0.68722188, -0.65417685, -1.11864391, -0.97661156, -0.623327  ,
       -0.9940236 ,  0.13284186,  0.45686198,  0.81551585,  0.4194784 ,
        1.11572889,  1.26644819,  0.97178619,  0.75026908, -1.40606238,
        0.02820316,  0.        ,  0.        ,  0.        ,  0.00380065,
        0.56223553, -0.89298856, -0.6630144 , -0.15196442, -0.56223553,
       -0.6511125 , -0.18628996, -0.34955759, -0.04722674, -0.03990117,
       -0.12579474, -0.50714322, -0.04888821, -0.50763857,  0.        ,
        0.        , -0.25813322, -0.19655276, -0.06565322, -0.02820316,
        0.        ,  0.        , -0.0437165 ,  0.        , -0.0437165 ,
        0.        ,  0.        ,  0.        , -0.11349173, -0.32

In [ ]:
def print_metrics(name, model, predictions, X_train, Y_train, Y_test):
    print("Results with "+name+":")
    print("Training set score: %f" % model.score(X_train, Y_train))
    print("Training set loss: %f" % model.loss_)
    percent = np.mean(predictions == Y_test)*100
    print("Accuracy:")
    print(percent)
    print("Accuracy (all): ", accuracy_score(predictions, Y_test))
    print("F1: ", 
        f1_score(predictions, Y_test, average=None), 
        f1_score(predictions, Y_test, average='micro')
    )
    print("R2: ", r2_score(predictions, Y_test))
    print("Precision: ", 
        precision_score(predictions, Y_test, average=None), 
        precision_score(predictions, Y_test, average='micro')
    )
    print("recall: ", 
        precision_score(predictions, Y_test, average=None), 
        recall_score(predictions, Y_test, average='micro')
    )

In [72]:
# http://scikit-learn.org/stable/modules/neural_networks_supervised.html
mlp = MLPClassifier(
    alpha=1e-5,
    hidden_layer_sizes=(71, 71),
    learning_rate='constant',
    solver='lbfgs'
)

mlp.fit(X_train, Y_train)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(71, 71), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='lbfgs', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [73]:
print_metrics("Multi-Layer Perceptron", mlp, mlp.predict(X_test), X_train, Y_train, Y_test)

Training set score: 0.606263
Training set loss: 1.058588


In [74]:
# probability score
X_train_probability = mlp.predict_proba(X_train) 
X_train_probability

array([[ 0.21280822,  0.307902  ,  0.21327221],
       [ 0.01173525,  0.00149825,  0.99399005],
       [ 0.40052878,  0.07664813,  0.33904097],
       ..., 
       [ 0.0060346 ,  0.94687078,  0.13733243],
       [ 0.02977759,  0.00880676,  0.98475146],
       [ 0.22427128,  0.17967391,  0.55204736]])

In [61]:
param_grid = {
    'hidden_layer_sizes': [(71,), (71, 71), (71, 71, 3)],
    'tol': [1e-2, 1e-3, 1e-4, 1e-5, 1e-6],
    'epsilon': [1e-3, 1e-7, 1e-8, 1e-9, 1e-8]
}
gridmlp = GridSearchCV(
    MLPClassifier(learning_rate='adaptive', learning_rate_init=1., early_stopping=True, shuffle=True),
    param_grid=param_grid, n_jobs=-1)
gridmlp.fit(X_train, Y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=True, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='adaptive',
       learning_rate_init=1.0, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'hidden_layer_sizes': [(71,), (71, 71), (71, 71, 3)], 'tol': [0.01, 0.001, 0.0001, 1e-05, 1e-06], 'epsilon': [0.001, 1e-07, 1e-08, 1e-09, 1e-08]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [62]:
gridmlp_predictions = gridmlp.predict(X_test)
print("Training set score: %f" % gridmlp.score(X_train, Y_train))
print("Training set loss: %f" % gridmlp.loss_)
print("Results with Multi-Layer Perceptron:")
percent = np.mean(gridmlp_predictions == Y_test)*100
print("Accuracy:")
print(percent)

print("accuracy_score: ", accuracy_score(gridmlp_predictions, Y_test))
print("r2_score: ", r2_score(gridmlp_predictions, Y_test))
# print("recal: ", recall_score(gridmlp_predictions, Y_test))

Results with Multi-Layer Perceptron:
Accuracy:
buy     77.205532
sell    77.634716
hold    54.840248
dtype: float64
accuracy_score:  0.548402479733
r2_score:  0.0


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/regression.py:482: DeprecationWarning: Default 'multioutput' behavior now corresponds to 'variance_weighted' value which is deprecated since 0.17, it will be changed to 'uniform_average' starting from 0.19.
  DeprecationWarning)


In [ ]:
# https://github.com/scikit-learn/scikit-learn/blob/master/examples/neural_networks/plot_mlp_training_curves.py

In [63]:
# https://stats.stackexchange.com/questions/260736/multiclass-classification-having-class-imbalance-with-gradient-boosting-classifi
# https://www.analyticsvidhya.com/blog/2016/02/complete-guide-parameter-tuning-gradient-boosting-gbm-python/

from sklearn.ensemble import GradientBoostingClassifier

# http://scikit-learn.org/stable/modules/neural_networks_supervised.html
gbc = GradientBoostingClassifier(
)

gbc.fit(X_train, Y_train)
gbc_predictions = gbc.predict(X_test)

print("Training set score: %f" % gbc.score(X_train, Y_train))
print("Training set loss: %f" % gbc.loss_)

print("Results with Gradient Boost:")
percent = np.mean(gbc_predictions == Y_test)*100
print("Accuracy:")
print(percent)
print("accuracy_score: ", accuracy_score(gbc_predictions, Y_test))
print("r2_score: ", r2_score(gbc_predictions, Y_test))
# print("recal: ", recall_score(X_test_predictions, Y_test))

ValueError: bad input shape (6291, 3)

In [70]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=100, min_samples_leaf=10,     
random_state=1)
rfc.fit(X_train, Y_train)
print(rfc.score)
#Accuracy of prediction
Y_pred = rfc.predict(X_test)
#Mean Standard Error
print('MSE:', mean_squared_error(Y_pred, Y_test))
print('score:', model.score(X_test, Y_test))


<bound method ClassifierMixin.score of RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=10,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=1, oob_score=False, random_state=1,
            verbose=0, warm_start=False)>
MSE: 0.296932125258
score: 0.388173581307


In [65]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import classification_report,f1_score


gb=GradientBoostingClassifier(n_estimators=200,learning_rate=.1)
gb.fit(X_train,Y_train)

cross_val_score(estimator=gb,X=X_test,y=Y_test,scoring='f1_weighted',cv=5)
print(classification_report(y_true=Y_test,Y_pred=gb.predict(X_test)))

/opt/conda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


ValueError: bad input shape (6291, 3)